In [1]:
# Import the dependencies.
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
import numpy as np
import re

# Import the CRSS connection string.
from config import crss_conn_string

# CONNECT TO THE DATABASE

In [2]:
# Create CSSR engine.
cssr_engine = create_engine(crss_conn_string);

In [3]:
# Connect to server.
cssr_conn = cssr_engine.connect();

In [4]:
# Create accidents, vehicles, people data frame.
avp_df = pd.read_sql("SELECT * FROM public.accident_vehicle_person", cssr_conn)

In [5]:
# Show first 10 rows.
pd.set_option("display.max_columns", None)
avp_df.head()

,casenum,urbancity,ve_total,ve_forms,permvit,num_inj,month,year,day_week,hour,alcohol,max_sev,wrk_zone,lgt_cond,weather,veh_no,numoccs,m_harmname,makename,tow_vehname,trav_speed,deformedname,towedname,speedrelname,vtrafwayname,vspd_lim,bdytyp_imname,mod_year,p_crash1name,per_no,rest_usename,rest_misname,helm_usename,helm_misname,drinkingname,alc_resname,drugsname,hospitalname,locationname,sex_imname,injsev_imname,peralch_imname,seat_imname,age_im
0,201901180165,1,2,2,3,0,1,2019,4,8,2,0,0,1,1,2,2,Motor Vehicle In-Transport,Toyota,No Trailing Units,0,Minor Damage,Not Reported,No,"Two-Way, Divided, Positive Median Barrier",98,5-door/4-door hatchback,2013,Stopped in Roadway,2,Restraint Used - Type Unknown,No Indication of Mis-Use,Not Applicable,None Used/Not Applicable,Not Reported,Test Not Given,Not Reported,Not Transported,Occupant of a Motor Vehicle,Female,No Apparent Injury (O),No (Alcohol Not Involved),"Second Seat, Right Side",87
1,201901180300,2,2,2,4,1,1,2019,4,10,2,1,0,1,1,1,2,Motor Vehicle In-Transport,Nissan/Datsun,No Trailing Units,998,No Damage,Not Towed,No,"Two-Way, Divided, Unprotected Median",30,"4-door sedan, hardtop",1999,Decelerating in Road,1,Shoulder and Lap Belt Used,No Indication of Mis-Use,Not Applicable,None Used/Not Applicable,No (Alcohol Not Involved),Test Not Given,No (drugs not involved),Not Transported,Occupant of a Motor Vehicle,Female,No Apparent Injury (O),No (Alcohol Not Involved),"Front Seat, Left Side",75
2,201901180300,2,2,2,4,1,1,2019,4,10,2,1,0,1,1,1,2,Motor Vehicle In-Transport,Nissan/Datsun,No Trailing Units,998,No Damage,Not Towed,No,"Two-Way, Divided, Unprotected Median",30,"4-door sedan, hardtop",1999,Decelerating in Road,2,Shoulder and Lap Belt Used,No Indication of Mis-Use,Not Applicable,None Used/Not Applicable,No (Alcohol Not Involved),Test Not Given,No (drugs not involved),Not Transported,Occupant of a Motor Vehicle,Female,No Apparent Injury (O),No (Alcohol Not Involved),"Front Seat, Right Side",75
3,201901182133,1,2,2,2,0,1,2019,5,13,2,0,0,1,1,1,1,Motor Vehicle In-Transport,Chevrolet,No Trailing Units,50,Minor Damage,Not Towed,No,"Two-Way, Divided, Positive Median Barrier",50,"4-door sedan, hardtop",2005,Changing Lanes,1,Shoulder and Lap Belt Used,No Indication of Mis-Use,Not Applicable,None Used/Not Applicable,No (Alcohol Not Involved),Not Reported,No (drugs not involved),Not Transported,Occupant of a Motor Vehicle,Female,No Apparent Injury (O),No (Alcohol Not Involved),"Front Seat, Left Side",90
4,201901182257,1,1,1,1,99,1,2019,5,99,9,9,0,9,2,1,99,"Harmful Event, Details Not Reported",Mercury,No Trailing Units,998,Disabling Damage,Towed Due to Disabling Damage,No,Entrance/Exit Ramp,35,"4-door sedan, hardtop",2009,Unknown,1,Reported as Unknown,None Used/Not Applicable,Not Applicable,None Used/Not Applicable,Reported as Unknown,Test Not Given,Reported as Unknown,Not Transported,Occupant of a Motor Vehicle,Reported as Unknown,Unknown/Not Reported,No (Alcohol Not Involved),"Front Seat, Left Side",998


In [6]:
# Write to CSV
avp_df.to_csv('all_gas_no_brakes.csv')

# PREPROCCESSING

In [7]:
# Describe Data
avp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256440 entries, 0 to 256439
Data columns (total 44 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   casenum         256440 non-null  object
 1   urbancity       256440 non-null  int64 
 2   ve_total        256440 non-null  int64 
 3   ve_forms        256440 non-null  int64 
 4   permvit         256440 non-null  int64 
 5   num_inj         256440 non-null  int64 
 6   month           256440 non-null  int64 
 7   year            256440 non-null  int64 
 8   day_week        256440 non-null  int64 
 9   hour            256440 non-null  int64 
 10  alcohol         256440 non-null  int64 
 11  max_sev         256440 non-null  int64 
 12  wrk_zone        256440 non-null  int64 
 13  lgt_cond        256440 non-null  int64 
 14  weather         256440 non-null  int64 
 15  veh_no          256440 non-null  int64 
 16  numoccs         256440 non-null  int64 
 17  m_harmname      256440 non-nu

In [8]:
# Recode Urban City : 0 = urban, 1 = city
avp_df['urbancity'] = avp_df['urbancity'].replace(1,0)
avp_df['urbancity'] = avp_df['urbancity'].replace(2,1)
avp_df['urbancity'].unique()

array([0, 1], dtype=int64)

In [9]:
# Recode Month : 1 = Jan, 12 = Dec
avp_df['month']= avp_df['month'].replace([1,2,3,4,5,6,7,8,9,10,11,12],['Jan','Feb','Mar','Apr','May',
                                                                             'Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
avp_df['month'].unique()

array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'], dtype=object)

In [10]:
# Recode Day of the week 1= Sunday, 7 = Saturday
avp_df['day_week']= avp_df['day_week'].replace([1,2,3,4,5,6,7],['Sun','Mon','Tues','Wed','Thus','Fri','Sat'])
avp_df['day_week'].unique()

array(['Wed', 'Thus', 'Sun', 'Fri', 'Mon', 'Tues', 'Sat'], dtype=object)

In [11]:
# Recode Alcohol - accident level 1= alcohol involved, 2,8,9 =  no alcohol
avp_df['alcohol'] = avp_df['alcohol'].replace([1,2,8,9],[1,0,0,0])
avp_df['alcohol'].unique()

array([0, 1], dtype=int64)

In [12]:
# Recode max severity - accident level 0,8,9 = none, 1,2,5 = minor, 3 = serious, 4,6 = fatal
avp_df['max_sev']=avp_df['max_sev'].replace([0,8,9,1,2,5,3,4,6],['none', 'none','none','minor','minor',
                                                                 'minor','serious','fatal','fatal'])
avp_df['max_sev'].unique()

array(['none', 'minor', 'serious', 'fatal'], dtype=object)

In [13]:
# Recode work zone 0 = no (0), 1,2,3,4 = yes (1)
avp_df['wrk_zone'] = avp_df['wrk_zone'].replace([0,1,2,3,4],[0,1,1,1,1])
avp_df['wrk_zone'].unique()

array([0, 1], dtype=int64)

In [14]:
# Recode lighting conditions 1= daylight, 2,3,6 = dark, 4=dawn 5=dusk, 7,8,9 = other
avp_df['lgt_cond']= avp_df['lgt_cond'].replace([1,2,3,6,4,5,7,8,9],['daylight','dark','dark','dark','dawn','dusk',
                                                                     'other','other','other'])
avp_df['lgt_cond'].unique()

array(['daylight', 'other', 'dark', 'dawn', 'dusk'], dtype=object)

In [15]:
# Recode weather 1 = clear 2,3 = rain/sleet/hail, 4,11 = snow, 5 = fog/smoke, 6=windy, 7=blowing dirt, 
# 10=cloudy, 12=freezing rain, 8, 98, 99 = other
avp_df['weather']=avp_df['weather'].replace([1,2,3,4,11,5,6,7,10,12,8,98,99],['clear','rain_sleet','rain_sleet',
                                                                              'snow_blowsnow','snow_blowsnow',
                                                                              'fog_smoke','windy','blowing_dirt','cloudy',
                                                                              'freezing_rain','other','other','other'])
avp_df['weather'].unique()

array(['clear', 'rain_sleet', 'cloudy', 'other', 'fog_smoke',
       'snow_blowsnow', 'windy', 'freezing_rain', 'blowing_dirt'],
      dtype=object)

In [16]:
avp_df['m_harmname'].unique()

array(['Motor Vehicle In-Transport',
       'Harmful Event, Details Not Reported', 'Tree (Standing Only)',
       'Parked Motor Vehicle', 'Pedestrian', 'Wall', 'Live Animal',
       'Pedalcyclist', 'Traffic Sign Support', 'Shrubbery',
       'Bridge Rail (Includes parapet)', 'Rollover/Overturn', 'Curb',
       'Embankment',
       'Motor Vehicle In-Transport Strikes or is Struck by Cargo, Persons or Objects Set-in-Motion from/by Another Motor Vehicle In Transport',
       'Cable Barrier', 'Fence', 'Utility Pole/Light Support', 'Ditch',
       'Guardrail Face', 'Culvert', 'Traffic Signal Support',
       'Concrete Traffic Barrier', 'Post, Pole or Other Supports',
       'Boulder', 'Fire Hydrant', 'Mail Box', 'Fell/Jumped from Vehicle',
       'Other Fixed Object', 'Non-Motorist on Personal Conveyance',
       'Other Object (not fixed)', 'Reported as Unknown',
       'Jackknife (harmful to this vehicle)', 'Guardrail End', 'Building',
       'Other Non-Collision',
       'Pavement Surface

In [17]:
# Recode most harmful event - vehicle level
avp_df['m_harmname'] = avp_df['m_harmname'].replace([
   'Motor Vehicle In-Transport',
    'Motor Vehicle in Motion Outside the Trafficway',
    'Working Motor Vehicle',
    
    'Parked Motor Vehicle', 
    
    'Pedalcyclist',
    'Pedestrian',
    'Non-Motorist on Personal Conveyance',
    'Ridden Animal or Animal Drawn Conveyance',
    'Live Animal',
    
    'Traffic Sign Support',
    'Utility Pole/Light Support',
    'Bridge Pier or Support',
    'Guardrail End',
    'Post, Pole or Other Supports',
    'Impact Attenuator/Crash Cushion',
    'Fire Hydrant',
    'Other Fixed Object',
    'Unknown Fixed Object',
    'Mail Box',
    'Traffic Signal Support',
    'Bridge Overhead Structure',
    'Building',
    
    'Bridge Rail (Includes parapet)',
    'Curb', 
    'Guardrail Face',
    'Concrete Traffic Barrier',
    'Other Traffic Barrier',
    'Wall', 
    'Cable Barrier',
    'Fence',
    
    'Shrubbery',
    'Tree (Standing Only)',
    'Boulder',
    'Snow Bank',
    
    'Embankment',
    'Ditch', 
    'Ground',
    'Culvert',
    
    'Fire/Explosion',
    
    'Motor Vehicle In-Transport Strikes or is Struck by Cargo, Persons or Objects Set-in-Motion from/by Another Motor Vehicle In Transport',
    'Object That Had Fallen From Motor Vehicle In-Transport',
    'Unknown Object Not Fixed',
    'Thrown or Falling Object',
    'Other Object (not fixed)',
    
    'Immersion or Partial Immersion',
    
    'Reported as Unknown',
    'Harmful Event, Details Not Reported',
    
    'Pavement Surface Irregularity (Ruts, Potholes, Grates, etc.)',
    'Jackknife (harmful to this vehicle)',
    'Other Non-Collision',
    'Cargo/Equipment Loss, Shift, or Damage [harmful]',
    'Rollover/Overturn',
    
    'Injured In Vehicle (Non-Collision)',
    'Fell/Jumped from Vehicle', 
    
     'Railway Vehicle',
    'Road Vehicle on Rails'    
],
[
    'harm_moving_veh',
    'harm_moving_veh',
    'harm_moving_veh',
    
    'harm_parked_veh',
    
    'harm_ped_animal',
    'harm_ped_animal',
    'harm_ped_animal',
    'harm_ped_animal',
    'harm_ped_animal',
    
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    
    'harm_nat_object',
    'harm_nat_object',
    'harm_nat_object',
    'harm_nat_object',
    
    'harm_terrain',
    'harm_terrain',
    'harm_terrain',
    'harm_terrain',
    
    'harm_fire',
    
    'harm_object',
    'harm_object',
    'harm_object',
    'harm_object',
    'harm_object',
    
    'harm_water',
    
    'harm_unknown',
    'harm_unknown',
    
    'harm_lost_control',
    'harm_lost_control',
    'harm_lost_control',
    'harm_lost_control',
    'harm_lost_control',
    
    'harm_injury_fallout',
    'harm_injury_fallout',
    
    'harm_train',
    'harm_train'    
])
avp_df['m_harmname'].unique()

array(['harm_moving_veh', 'harm_unknown', 'harm_nat_object',
       'harm_parked_veh', 'harm_ped_animal', 'harm_barrier',
       'harm_fixed_manmade', 'harm_lost_control', 'harm_terrain',
       'harm_object', 'harm_injury_fallout', 'harm_fire', 'harm_water',
       'harm_train'], dtype=object)

In [18]:
avp_df['makename'].unique()

array(['Toyota', 'Nissan/Datsun', 'Chevrolet', 'Mercury', 'AM General',
       'Dodge', 'Mazda', 'Ford', 'Honda', 'Lexus', 'GMC', 'Subaru',
       'Buick / Opel', 'KIA', 'Pontiac', 'Acura', 'Mercedes-Benz',
       'Kenworth', 'Hyundai', 'Unknown Make', 'Cadillac', 'Audi',
       'Mitsubishi', 'International Harvester/Navistar', 'Gillig',
       'Other Make', 'Saturn', 'Chrysler', 'Lincoln', 'Infiniti',
       'Volkswagen', 'Jeep / Kaiser-Jeep / Willys- Jeep', 'Scion',
       'Harley-Davidson', 'BMW', 'Freightliner', 'Suzuki', 'Bluebird',
       'Peterbilt', 'Volvo', 'Land Rover', 'Yamaha', 'Kawasaki',
       'Not Reported', 'Porsche', 'Isuzu', 'Triumph', 'Grumman',
       'Other Import', 'Mack', 'Oldsmobile',
       'Other Domestic Manufacturers', 'Jaguar', 'Saab', 'Smart', 'MCI',
       'Fiat', 'Ducati', 'Victory', 'Plymouth', 'White/Autocar White/GMC',
       'Eagle', 'Thomas Built', 'Alfa Romeo', 'Moto-Guzzi', 'Daewoo',
       'American Motors'], dtype=object)

In [19]:
# Recode vehicle make as make_country
avp_df['make_country'] = avp_df['makename'].replace([
    'Toyota',
    'Honda',
    'Subaru',
    'Nissan/Datsun',
    'Acura',
    'Suzuki',
    'Lexus',
    'Mazda', 
    'Mitsubishi', 
    'Infiniti', 
    'Isuzu',
    'Scion',
    
    'KIA',
    'Daewoo',
    'Hyundai',
    
    'Chevrolet', 
    'Ford',
    'Pontiac',
    'Cadillac',
    'Dodge', 
    'Chrysler', 
    'GMC',
    'Jeep / Kaiser-Jeep / Willys- Jeep', 
    'Buick / Opel',
    'Other Domestic Manufacturers', 
    'Lincoln',
    'Oldsmobile', 
    'Mercury',
    'Plymouth', 
    'Eagle', 
    'American Motors',
    'Saturn',
    
    'Freightliner', 
    'AM General', 
    'International Harvester/Navistar',
    'Peterbilt',
    'Mack',
    'Kenworth',
    'Thomas Built', 
    'Bluebird',
    'White/Autocar White/GMC',
    'Gillig', 
    'MCI',
    'Grumman',
    
    'Mercedes-Benz',
    'Volkswagen',
    'Audi',
    'BMW', 
    'Smart',
    'Porsche',
    
    'Victory',
    
    'Volvo',
    'Saab', 
    
    'Ducati', 
    'Harley-Davidson', 
    'Yamaha', 
    'Kawasaki',
    'Moto-Guzzi',
    
    'Jaguar',
    'Land Rover',
    'Triumph', 
    
    'Alfa Romeo',
    'Fiat',
    
    'Other Import',
    'Other Make',
    'Unknown Make',
    'Not Reported',
],
[
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    
    'make_Korea',
    'make_Korea',
    'make_Korea',
    
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    
    'make_Germany',
    'make_Germany',
    'make_Germany',
    'make_Germany',
    'make_Germany',
    'make_Germany',
    
    'make_China',
    
    'make_Sweden',
    'make_Sweden',
    
    'make_motorcycle',
    'make_motorcycle',
    'make_motorcycle',
    'make_motorcycle',
    'make_motorcycle',
    
    'make_England',
    'make_England',
    'make_England',
    
    'make_Italy',
    'make_Italy',
    
    'make_other',
    'make_other',
    'make_other',
    'make_other',    
])
avp_df['make_country'].unique()

array(['make_Japan', 'make_US', 'make_US_truck', 'make_Korea',
       'make_Germany', 'make_other', 'make_motorcycle', 'make_Sweden',
       'make_England', 'make_Italy', 'make_China'], dtype=object)

In [20]:
# Recode tow_vehname - towing a vehichle 0 = no, 1 = yes
avp_df['tow_vehname']=avp_df['tow_vehname'].replace([
    'No Trailing Units',
    'One Trailing Unit',
    'Unknown',
    'Two Trailing Units',
    'Vehicle Towing Another Motor Vehicle - Non-Fixed Linkage',
    'Three or More Trailing Units',
    'Vehicle Towing Another Motor Vehicle - Fixed Linkage',
    'Yes, Number of Trailing Units Unknown'
],
[0,1,1,1,1,1,1,1])
avp_df['tow_vehname'].unique()

array([0, 1], dtype=int64)

In [21]:
# Determine median speed for imputing for unknown travel Speed
avp_df['trav_speed_temp'] = avp_df['trav_speed']
avp_df['trav_speed_temp'] = avp_df['trav_speed_temp'].replace([997,998,999],[np.NaN,np.NaN,np.NaN])
avp_df['trav_speed_temp'].describe()

count    126350.000000
mean         22.381559
std          22.198566
min           0.000000
25%           0.000000
50%          15.000000
75%          40.000000
max         150.000000
Name: trav_speed_temp, dtype: float64

In [22]:
# Replace unknown travel speed with median travel speed 
avp_df['trav_speed']=avp_df['trav_speed'].replace([997,998,999],[155,23,23])
avp_df['trav_speed'].describe()

count    256440.000000
mean         22.697348
std          15.593666
min           0.000000
25%          15.000000
50%          23.000000
75%          23.000000
max         155.000000
Name: trav_speed, dtype: float64

In [23]:
# Recode deformedname
avp_df['deformedname']=avp_df['deformedname'].replace([
    'Not Reported',
    'Reported as Unknown'
],
[
    'Minor Damage',
    'Minor Damage'
])
avp_df['deformedname'].unique()

array(['Minor Damage', 'No Damage', 'Disabling Damage',
       'Functional Damage'], dtype=object)

In [24]:
# Recode towed Name 0 = no 1 = yes
avp_df['towedname']=avp_df['towedname'].replace([
    'Towed Due to Disabling Damage',
    'Not Towed',
    'Towed Not Due to Disabling Damage',
    'Towed, Unknown Reason',
    'Not Reported',
    'Towed But Not Due to Disabling Damage',
    'Reported as Unknown'
],[1,0,1,1,0,1,0])
avp_df['towedname'].unique()

array([0, 1], dtype=int64)

In [25]:
avp_df['speedrelname'].unique()

array(['No', 'Reported as Unknown', 'Yes, Too Fast for Conditions',
       'Yes, Specifics Unknown', 'Yes, Exceeded Speed Limit',
       'No Driver Present/Unknown if Driver Present', 'Yes, Racing'],
      dtype=object)

In [26]:
# Recode speed related 0 = n 1 = y
avp_df['speedrelname']=avp_df['speedrelname'].replace([
    'No', 
    'Yes, Too Fast for Conditions',
    'Reported as Unknown',
    'Yes, Exceeded Speed Limit',
    'Yes, Specifics Unknown',
    'No Driver Present/Unknown if Driver Present', 
    'Yes, Racing'
],[0,1,0,1,1,0,1])
avp_df['speedrelname'].unique()

array([0, 1], dtype=int64)

In [27]:
avp_df['vtrafwayname'].unique()

array(['Two-Way,  Divided, Positive  Median Barrier',
       'Two-Way, Divided, Unprotected Median', 'Entrance/Exit Ramp',
       'Two-Way, Not Divided With a Continuous Left-Turn Lane',
       'Two-Way, Not Divided', 'Non-Trafficway or Driveway Access',
       'One-Way Trafficway', 'Not Reported', 'Reported as Unknown'],
      dtype=object)

In [28]:
# Recode traffic way name 
avp_df['vtrafwayname']=avp_df['vtrafwayname'].replace([
    'Two-Way, Divided, Unprotected Median',
    'Not Reported',
    'Two-Way,  Divided, Positive  Median Barrier',
    'Two-Way, Not Divided',
    'Two-Way, Not Divided With a Continuous Left-Turn Lane',
    'Non-Trafficway or Driveway Access',
    'Entrance/Exit Ramp',
    'One-Way Trafficway', 
    'Reported as Unknown'
],
[
    'Two_way_div_med_nobar',
    'Two_way',
    'Two_way_div_med_bar',
    'Two_way',
    'Two-way',
    'Parking_lot_driveway',
    'Exit_on_ramp',
    'One-way',
    'Two-way'
])
avp_df['vtrafwayname'].unique()

array(['Two_way_div_med_bar', 'Two_way_div_med_nobar', 'Exit_on_ramp',
       'Two-way', 'Two_way', 'Parking_lot_driveway', 'One-way'],
      dtype=object)

In [29]:
avp_df['vspd_lim'].value_counts()

45    47465
35    42468
98    34694
40    24797
55    23671
25    20691
30    15468
65    12710
70    11755
50     7727
0      6416
60     3798
20     1461
15     1445
75      681
10      466
80      303
99      241
5       182
90        1
Name: vspd_lim, dtype: int64

In [30]:
# Recode speed limits (impute with the mode = 45)
avp_df['vspd_lim']=avp_df['vspd_lim'].replace([98,99],[45,45])
avp_df['vspd_lim'].value_counts()

45    82400
35    42468
40    24797
55    23671
25    20691
30    15468
65    12710
70    11755
50     7727
0      6416
60     3798
20     1461
15     1445
75      681
10      466
80      303
5       182
90        1
Name: vspd_lim, dtype: int64

In [31]:
avp_df['bdytyp_imname'].unique()

array(['5-door/4-door hatchback', '4-door sedan, hardtop',
       'Compact Utility (Utility Vehicle Categories Small and Midsize)',
       'Light Pickup',
       'Minivan (Chrysler Town and Country, Caravan, Grand Caravan, Voyager, Voyager, Honda-Odyssey, ...)',
       'Large utility (ANSI D16.1 Utility Vehicle Categories and Full Size and Large)',
       'Station Wagon (excluding van and truck based)',
       'Single-unit straight truck or Cab-Chassis (GVWR greater than 26,000 lbs.)',
       'Unknown body type',
       'Utility station wagon (includes suburban limousines, Suburban, Travellall, Grand Wagoneer)',
       '2-door sedan,hardtop,coupe',
       'Truck-tractor (Cab only, or with any number of trailing unit; any weight)',
       'Transit Bus (City Bus)', 'Moped or motorized bicycle',
       'Motor Scooter', 'Other or Unknown automobile type',
       'Sedan/Hardtop, number of doors unknown',
       '3-door/2-door hatchback', 'Convertible(excludes sun-roof,t-bar)',
       'Two W

In [32]:
# Recode body type
avp_df['bdytyp_imname']=avp_df['bdytyp_imname'].replace([
    '4-door sedan, hardtop',
    '5-door/4-door hatchback', 
    'Sedan/Hardtop, number of doors unknown', 
    'Hatchback, number of doors unknown',
    'Auto-based pickup (includes E1 Camino, Caballero, Ranchero, SSR, G8-ST, Subaru Brat, Rabbit Pickup)',
    'Large Limousine-more than four side doors or stretched chassis',
    'Auto-based panel (cargo station wagon, auto-based ambulance or hearse)',
    'Station Wagon (excluding van and truck based)',
    'Other or Unknown automobile type',
    'Unknown body type', 
    'Not Reported',
    
    '2-door sedan,hardtop,coupe',
    '3-door/2-door hatchback',
    '3-door coupe',
    
    'Convertible(excludes sun-roof,t-bar)',
    
    'Light Pickup',
    'Unknown light truck type', 
    'Unknown (pickup style) light conventional truck type',
    'Unknown light vehicle type (automobile,utility vehicle, van, or light truck)',
    'Other light conventional truck type',
    'Compact Utility (Utility Vehicle Categories Small and Midsize)', 
    'Cab Chassis Based (includes Rescue Vehicle, Light Stake, Dump, and Tow Truck)',
    
    'Minivan (Chrysler Town and Country, Caravan, Grand Caravan, Voyager, Voyager, Honda-Odyssey, ...)',
    'Large Van-Includes van-based buses (B150-B350, Sportsman, Royal Maxiwagon, Ram, Tradesman,...)',
    'Van-Based Bus GVWR greater than 10,000 lbs.', 
    'Unknown van type',
    'Step van (GVWR greater than 10,000 lbs.)',
    'Other van type (Hi-Cube Van, Kary)', 
    'Step-van or walk-in van (GVWR less than or equal to 10,000 lbs.)',
    
    'Large utility (ANSI D16.1 Utility Vehicle Categories and Full Size and Large)',
    'Medium/heavy Pickup (GVWR greater than 10,000 lbs.)',
    'Utility Vehicle, Unknown body type', 
    'Utility station wagon (includes suburban limousines, Suburban, Travellall, Grand Wagoneer)',
    'Unknown truck type (light/medium/heavy)',
    
    'Truck-tractor (Cab only, or with any number of trailing unit; any weight)',
    'Single-unit straight truck or Cab-Chassis (GVWR range 19,501 to 26,000 lbs.)',
    'Single-unit straight truck or Cab-Chassis (GVWR unknown)',
    'Unknown medium/heavy truck type', 
    'Single-unit straight truck or Cab-Chassis (GVWR range 10,001 to 19,500 lbs.)',
    'Single-unit straight truck or Cab-Chassis (GVWR greater than 26,000 lbs.)',
    'Unknown if single-unit or combination unit Medium Truck (GVWR range 10,001 lbs. to 26,000 lbs.)',
    'Unknown if single-unit or combination unit Heavy Truck (GVWR greater than 26,000 lbs.)',
    
    'Two Wheel Motorcycle (excluding motor scooters)',
    'Motor Scooter',
    'Moped or motorized bicycle',
    'Unknown motored cycle type',
    'Off-road Motorcycle', 
    'Unenclosed Three Wheel Motorcycle / Unenclosed Autocycle (1 Rear Wheel)',
    'Three-wheel Motorcycle (2 Rear Wheels)',
    'Other motored cycle type (mini-bikes, pocket motorcycles pocket bikes)',
    'Unknown Three Wheel Motorcycle Type',
    
    'School Bus',
    'Transit Bus (City Bus)',
    'Other Bus Type', 
    'Unknown Bus Type',
    'Cross Country/Intercity Bus',
    
    'Construction equipment other than trucks (includes graders)',  
    'Farm equipment other than trucks',
    
    'Medium/heavy truck based motorhome',
    'Medium/Heavy Vehicle Based Motor Home',
    'Camper or motorhome, unknown truck type',
    'Light Vehicle Based Motor Home (chassis mounted)',
    'Light Truck Based Motorhome (Chassis Mounted)',
    
    'Recreational Off-Highway Vehicle',
    'Other vehicle type (includes go-cart, fork-lift, city street sweeper dunes/swamp buggy)',
    'Low Speed Vehicle (LSV) / Neighborhood Electric Vehicle (NEV)',
    'ATV/ATC [All-Terrain Cycle]',
    'Golf Cart',
     
     'Snowmobile'
    
],[
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    
    '2_door_sedan',
    '2_door_sedan',
    '2_door_sedan',
    
    'Convertable',
    
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    
    'Van',
    'Van',
    'Van',
    'Van',
    'Van',
    'Van',
    'Van',
    
    'Large_SUV',
    'Large_SUV',
    'Large_SUV',
    'Large_SUV',
    'Large_SUV',
    
    'Truck',
    'Truck',
    'Truck',
    'Truck',
    'Truck',
    'Truck',
    'Truck',
    'Truck',
    
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    
    'Bus',
    'Bus',
    'Bus',
    'Bus',
    'Bus',
    
    'Construction_farm_equip',
    'Construction_farm_equip',
    
    'Motorhome_RV',
    'Motorhome_RV',
    'Motorhome_RV',
    'Motorhome_RV',
    'Motorhome_RV',
    
    'ATV_rec_vehicle',
    'ATV_rec_vehicle',
    'ATV_rec_vehicle',
    'ATV_rec_vehicle',
    'ATV_rec_vehicle',
    
     'Snowmobile',
    
])
avp_df['bdytyp_imname'].unique()


array(['4_door_sedan', 'Small_SUV_light_truck', 'Van', 'Large_SUV',
       'Truck', '2_door_sedan', 'Bus', 'Motorcylcle_trike', 'Convertable',
       'Motorhome_RV', 'Construction_farm_equip', 'ATV_rec_vehicle',
       'Snowmobile'], dtype=object)

In [33]:
avp_df['p_crash1name'].unique()

array(['Stopped in Roadway', 'Decelerating in Road', 'Changing Lanes',
       'Unknown', 'Going Straight', 'Turning Left', 'Negotiating a Curve',
       'Turning Right', 'Starting in Road',
       'Passing or Overtaking Another Vehicle',
       'Successful Avoidance Maneuver to a Previous Critical Event',
       'Backing Up (other than for Parking Position)', 'Other(specify:)',
       'No Driver Present / Unknown if Driver Present', 'Making a U-turn',
       'Leaving a Parking Position', 'Merging', 'Accelerating in Road',
       'Disabled or Parked in Travel lane', 'Entering a Parking Position'],
      dtype=object)

In [34]:
# Recode prior to crash movement: some not binned. unknowns binned with going straight
avp_df['p_crash1name'] = avp_df['p_crash1name'].replace([
    'Going Straight', 
    'Other(specify:)',
    'Unknown', 
    'No Driver Present / Unknown if Driver Present',
        
    'Decelerating in Road',
    'Stopped in Roadway',
    'Backing Up (other than for Parking Position)',
    
    'Starting in Road',
    'Accelerating in Road', 
    
    'Disabled or Parked in Travel lane', 
    'Entering a Parking Position',
    'Leaving a Parking Position',
     
]
,[
    'Going straight',
    'Going straight',
    'Going straight',
    'Going straight',
    
    'Stopping_backup',
    'Stopping_backup',
    'Stopping_backup',
    
    'Start on road',
    'Start on road',
    
    'Diasbled_parked',
    'Diasbled_parked',
    'Diasbled_parked',
])
avp_df['p_crash1name'].unique()

array(['Stopping_backup', 'Changing Lanes', 'Going straight',
       'Turning Left', 'Negotiating a Curve', 'Turning Right',
       'Start on road', 'Passing or Overtaking Another Vehicle',
       'Successful Avoidance Maneuver to a Previous Critical Event',
       'Making a U-turn', 'Diasbled_parked', 'Merging'], dtype=object)

In [35]:
# Recode restrictive device
avp_df['rest_usename'] = avp_df['rest_usename'].replace([
    'Shoulder and Lap Belt Used',
    'Shoulder Belt Only Used', 
    'Other',
    'Restraint Used - Type Unknown',
    'Lap Belt Only Used',
    'Reported as Unknown',
    'Not Reported',
    
    'None Used/Not Applicable', 

    'Child Restraint Type Unknown', 
    'Booster Seat', 
    'Child Restraint System - Forward Facing',
    'Child Restraint System  - Rear Facing',
    'Child Restraint System - Rear Facing',
    
    'Racing-Style Harness Used'
    
   
],[
    'Seatbelt',
    'Seatbelt',
    'Seatbelt',
    'Seatbelt',
    'Seatbelt',
    'Seatbelt',
    'Seatbelt',
    
    'No_seatbelt',
    
    'Child_restraint',
    'Child_restraint',
    'Child_restraint',
    'Child_restraint',
    'Child_restraint',
    
    'Harness'
])

avp_df['rest_usename'].unique()

array(['Seatbelt', 'No_seatbelt', 'Child_restraint', 'Harness'],
      dtype=object)

In [36]:
# Recode retrain misuse 0 = no, 1 = yes
avp_df['rest_misname'] = avp_df['rest_misname'].replace([
    'No Indication of Mis-Use', 
    'None Used/Not Applicable',
    'Yes, Indication of Mis-Use'
],[0,0,1])
avp_df['rest_misname'].unique()

array([0, 1], dtype=int64)

In [37]:
# Recode helmet wear 0= no, 1=yes
avp_df['helm_usename']=avp_df['helm_usename'].replace([
    'Not Applicable',
    'Helmet, Other than DOT-Compliant Motorcycle Helmet',
    'Helmet, Unknown if DOT-Compliant', 
    'No Helmet', 
    'Not Reported',
    'DOT-Compliant Motorcycle Helmet',
    'Reported as Unknown if Helmet Worn'
],[0,1,1,0,0,1,0])
avp_df['helm_usename'].unique()

array([0, 1], dtype=int64)

In [38]:
# Recode helmet misuse 0 = no 1 = yes
avp_df['helm_misname']=avp_df['helm_misname'].replace([
    'None Used/Not Applicable', 
    'No Indication of Mis-Use',
    'Yes, Indication of Mis-Use'
],[0,0,1])
avp_df['helm_misname'].unique()

array([0, 1], dtype=int64)

In [39]:
# Recode police reported drinking 0 = no 1 = yes
avp_df['drinkingname'] = avp_df['drinkingname'].replace([
    'No (Alcohol Not Involved)', 
    'Not Reported', 
    'Reported as Unknown',
    'Yes (Alcohol Involved)'
],[0,0,0,1])
avp_df['drinkingname'].unique()

array([0, 1], dtype=int64)

In [40]:
avp_df['alc_resname'].unique()


array(['Test Not Given', 'Not Reported',
       'AC Test Performed, Results Unknown', '0.040 % BAC',
       'Reported as Unknown if Tested', '0.209 % BAC', '0.172 % BAC',
       '0.000 % BAC', '0.160 % BAC', '0.270 % BAC', '0.121 % BAC',
       '0.082 % BAC', '0.236 % BAC', '0.101 % BAC', '0.150 % BAC',
       '0.395 % BAC', '0.110 % BAC', '0.230 % BAC', '0.182 % BAC',
       '0.184 % BAC', '0.171 % BAC', '0.158 % BAC', '0.041 % BAC',
       '0.140 % BAC', '0.003 % BAC', '0.197 % BAC', '0.126 % BAC',
       '0.180 % BAC', '0.181 % BAC', '0.060 % BAC',
       'Positive Reading with No Actual Value', '0.157 % BAC',
       '0.300 % BAC', '0.212 % BAC', '0.220 % BAC', '0.375 % BAC',
       '0.195 % BAC', '0.187 % BAC', '0.208 % BAC', '0.088 % BAC',
       '0.194 % BAC', '0.090 % BAC', '0.017 % BAC', '0.070 % BAC',
       '0.011 % BAC', '0.107 % BAC', '0.130 % BAC', '0.100 % BAC',
       '0.080 % BAC', '0.014 % BAC', '0.290 % BAC', '0.078 % BAC',
       '0.030 % BAC', '0.210 % BAC', '0.165 

In [41]:
avp_df['alc_resname_08']=avp_df['alc_resname'].replace([
    
    'Test Not Given', 'Not Reported', 'Reported as Unknown if Tested',
       'AC Test Performed, Results Unknown', '0.220 % BAC', '0.000 % BAC',
       '0.070 % BAC', '0.176 % BAC', '0.200 % BAC', '0.080 % BAC',
       '0.244 % BAC', '0.170 % BAC', '0.020 % BAC', '0.125 % BAC',
       '0.216 % BAC', '0.140 % BAC', '0.090 % BAC', '0.190 % BAC',
       '0.230 % BAC', '0.150 % BAC',
       'Positive Reading with No Actual Value', '0.172 % BAC',
       '0.160 % BAC', '0.187 % BAC', '0.100 % BAC', '0.110 % BAC',
       '0.181 % BAC', '0.157 % BAC', '0.210 % BAC', '0.158 % BAC',
       '0.119 % BAC', '0.180 % BAC', '0.189 % BAC', '0.130 % BAC',
       '0.030 % BAC', '0.208 % BAC', '0.245 % BAC', '0.144 % BAC',
       '0.121 % BAC', '0.280 % BAC', '0.195 % BAC', '0.194 % BAC',
       '0.017 % BAC', '0.011 % BAC', '0.050 % BAC', '0.076 % BAC',
       '0.116 % BAC', '0.060 % BAC', '0.072 % BAC', '0.221 % BAC',
       '0.193 % BAC', '0.296 % BAC', '0.161 % BAC', '0.255 % BAC',
       '0.075 % BAC', '0.155 % BAC', '0.229 % BAC', '0.214 % BAC',
       '0.152 % BAC', '0.250 % BAC', '.94 % or Greater', '0.238 % BAC',
       '0.123 % BAC', '0.310 % BAC', '0.014 % BAC', '0.275 % BAC',
       '0.290 % BAC', '0.188 % BAC', '0.137 % BAC', '0.114 % BAC',
       '0.040 % BAC', '0.212 % BAC', '0.078 % BAC', '0.184 % BAC',
       '0.375 % BAC', '0.178 % BAC', '0.088 % BAC', '0.165 % BAC',
       '0.115 % BAC', '0.209 % BAC', '0.300 % BAC', '0.168 % BAC',
       '0.132 % BAC', '0.260 % BAC', '0.318 % BAC', '0.185 % BAC',
       '0.286 % BAC', '0.173 % BAC', '0.077 % BAC', '0.186 % BAC',
       '0.120 % BAC', '0.167 % BAC', '0.162 % BAC', '0.111 % BAC',
       '0.129 % BAC', '0.287 % BAC', '0.081 % BAC', '0.302 % BAC',
       '0.094 % BAC', '0.240 % BAC', '0.135 % BAC', '0.142 % BAC',
       '0.131 % BAC', '0.148 % BAC', '0.128 % BAC', '0.309 % BAC',
       '0.026 % BAC', '0.159 % BAC', '0.196 % BAC', '0.191 % BAC',
       '0.151 % BAC', '0.270 % BAC', '0.108 % BAC', '0.105 % BAC',
       '0.204 % BAC', '0.016 % BAC', '0.057 % BAC', '0.109 % BAC',
       '0.093 % BAC', '0.113 % BAC', '0.307 % BAC', '0.085 % BAC',
       '0.008 % BAC', '0.164 % BAC', '0.269 % BAC', '0.320 % BAC',
       '0.138 % BAC', '0.146 % BAC', '0.062 % BAC', '0.083 % BAC',
       '0.177 % BAC', '0.247 % BAC', '0.175 % BAC', '0.223 % BAC',
       '0.102 % BAC', '0.028 % BAC', '0.222 % BAC', '0.126 % BAC',
       '0.228 % BAC', '0.336 % BAC', '0.920 % BAC', '0.239 % BAC',
       '0.232 % BAC', '0.201 % BAC', '0.281 % BAC', '0.001 % BAC',
       '0.285 % BAC', '0.010 % BAC', '0.197 % BAC', '0.249 % BAC',
       '0.101 % BAC', '0.361 % BAC', '0.395 % BAC', '0.163 % BAC',
       '0.096 % BAC', '0.382 % BAC', '0.271 % BAC', '0.350 % BAC',
       '0.182 % BAC', '0.047 % BAC', '0.211 % BAC', '0.154 % BAC',
       '0.166 % BAC', '0.053 % BAC', '0.042 % BAC', '0.218 % BAC',
       '0.192 % BAC', '0.215 % BAC', '0.004 % BAC', '0.174 % BAC',
       '0.231 % BAC', '0.226 % BAC', '0.227 % BAC', '0.099 % BAC',
       '0.139 % BAC', '0.082 % BAC', '0.124 % BAC', '0.002 % BAC',
       '0.031 % BAC', '0.272 % BAC', '0.213 % BAC', '0.058 % BAC',
       '0.045 % BAC', '0.262 % BAC', '0.198 % BAC', '0.378 % BAC',
       '0.224 % BAC', '0.041 % BAC', '0.390 % BAC', '0.112 % BAC',
       '0.117 % BAC', '0.205 % BAC', '0.305 % BAC', '0.298 % BAC',
       '0.179 % BAC', '0.289 % BAC', '0.242 % BAC', '0.012 % BAC',
       '0.147 % BAC', '0.243 % BAC', '0.248 % BAC', '0.106 % BAC',
       '0.234 % BAC', '0.141 % BAC', '0.274 % BAC', '0.143 % BAC',
       '0.256 % BAC', '0.145 % BAC', '0.006 % BAC', '0.202 % BAC',
       '0.207 % BAC', '0.054 % BAC', '0.380 % BAC', '0.068 % BAC',
       '0.067 % BAC', '0.384 % BAC', '0.027 % BAC', '0.253 % BAC',
       '0.156 % BAC', '0.292 % BAC', '0.136 % BAC', '0.254 % BAC',
       '0.322 % BAC', '0.206 % BAC', '0.237 % BAC', '0.600 % BAC',
       '0.252 % BAC', '0.095 % BAC', '0.340 % BAC', '0.048 % BAC',
       '0.153 % BAC', '0.294 % BAC', '0.337 % BAC', '0.091 % BAC',
       '0.233 % BAC', '0.241 % BAC', '0.043 % BAC', '0.171 % BAC',
       '0.023 % BAC', '0.334 % BAC', '0.520 % BAC', '0.007 % BAC',
       '0.069 % BAC', '0.059 % BAC', '0.104 % BAC', '0.079 % BAC',
       '0.268 % BAC', '0.013 % BAC', '0.107 % BAC', '0.236 % BAC',
       '0.217 % BAC', '0.066 % BAC', '0.098 % BAC', '0.203 % BAC',
       '0.251 % BAC', '0.003 % BAC', '0.308 % BAC', '0.278 % BAC',
       '0.246 % BAC', '0.064 % BAC', '0.235 % BAC', '0.086 % BAC',
       '0.061 % BAC', '0.259 % BAC', '0.169 % BAC'
],[
   0,0,0,
    0,1,0,
    0,1,1,1,
    1,1,0,1,
    1,1,1,1,
    1,1,
    0,1,
    1,1,1,1,
    1,1,1,1,
    1,1,1,1,
    0,1,1,1,
    1,1,1,1,
    0,0,0,0,
    1,0,0,1,
    1,1,1,1,
    0,1,1,1,
    1,1,1,1,
    1,1,0,1,
    1,1,1,1,
    0,1,0,1,
    1,1,1,1,
    1,1,1,1,
    1,1,1,1,
    1,1,0,1,
    1,1,1,1,
    1,1,1,1,
    1,1,1,1,
    1,1,1,1,
    0,1,1,1,
    1,1,1,1,
    1,0,0,1,
    1,1,1,1,
    0,1,1,1,
    1,1,0,1,
    1,1,1,1,
    1,0,1,1,
    1,1,1,1,
    1,1,1,0,
    1,0,1,1,
    1,1,1,1,
    1,1,1,1,
    1,0,1,1,
    1,0,0,1,
    1,1,0,1,
    1,1,1,1,
    1,1,1,0,
    0,1,1,0,
    0,1,1,1,
    1,0,1,1,
    1,1,1,1,
    1,1,1,0,
    1,1,1,1,
    1,1,1,1,
    1,1,0,1,
    1,0,1,0,
    0,1,0,1,
    1,1,1,1,
    1,1,1,1,
    1,1,1,0,
    1,1,1,1,
    1,1,0,1,
    0,1,1,0,
    0,0,1,0,
    1,0,1,1,
    1,0,1,1,
    1,0,1,1,
    1,0,1,1,
    0,1,1   
])
avp_df['alc_resname_08'].unique()

array([0, 1], dtype=int64)

In [42]:
# Recode drugs involved 0 = no, 1 = yes
avp_df['drugsname']=avp_df['drugsname'].replace([
    'No (drugs not involved)', 
    'Not Reported', 
    'Reported as Unknown',
    'Yes (drugs involved)'
],[0,0,0,1])
avp_df['drugsname'].unique()

array([0, 1], dtype=int64)

In [43]:
# recode transfer to hospital 0 = no 1 = yes
avp_df['hospitalname']=avp_df['hospitalname'].replace([
    'EMS Ground', 
    'Not Transported', 
    'Other', 
    'EMS Air',
    'EMS Unknown Mode', 
    'Reported as Unknown',
    'Transported  Unknown Source', 
    'Not Reported', 
    'Law Enforcement',
    'Not Transported for Treatment'
],[1,0,1,1,1,0,1,0,1,0])
avp_df['hospitalname'].unique()

array([0, 1], dtype=int64)

In [44]:
avp_df['locationname'].unique()
# NOT USEFUL

array(['Occupant of a Motor Vehicle'], dtype=object)

In [45]:
avp_df['sex_imname'].value_counts()

Male                   135342
Female                 109803
Not Reported             7708
Reported as Unknown      3587
Name: sex_imname, dtype: int64

In [46]:
# Recode sex 0 = female, 1 = male : Assign not reported as male, unknown as female
avp_df['sex_imname']=avp_df['sex_imname'].replace([
    'Female', 
    'Male', 
    'Reported as Unknown', 
    'Not Reported'
],[0,1,0,1])

avp_df['sex_imname'].value_counts()

1    143050
0    113390
Name: sex_imname, dtype: int64

In [47]:
# Recode person level of injury
avp_df['injsev_imname'] = avp_df['injsev_imname'].replace([
    'Suspected Serious Injury (A)', 
    'No Apparent Injury (O)',
    'Possible Injury (C)', 
    'Unknown/Not Reported',
    'Suspected Minor Injury (B)',
    'Fatal Injury (K)',
    'Injured, Severity Unknown', 
    'Died Prior to Crash*'
],[
    'serious',
    'none',
    'minor',
    'none',
    'minor',
    'fatal',
    'minor',
    'fatal'
 ])
avp_df['injsev_imname'].unique()

array(['none', 'serious', 'minor', 'fatal'], dtype=object)

In [48]:
# Recode personal alcohol use
avp_df['peralch_imname']=avp_df['peralch_imname'].replace([
    'No (Alcohol Not Involved)',
    'Yes (Alcohol Involved)'
],[0,1])
avp_df['peralch_imname'].unique()

array([0, 1], dtype=int64)

In [49]:
avp_df['seat_imname'].unique()

array(['Second Seat, Right Side', 'Front Seat, Left Side',
       'Front Seat, Right Side', 'Second Seat, Left Side',
       'Front Seat, Middle', 'Reported as Unknown', 'Not Reported',
       'Front Seat, Unknown', 'Second Seat, Middle',
       'Appended to a Motor Vehicle for Motion', 'Third Seat, Right Side',
       'Third Seat, Left Side', 'Sleeper Section of Cab (Truck)',
       'Third Seat, Middle', 'Second Seat, Unknown',
       'Other Passenger in enclosed passenger or cargo area',
       'Front Seat, Other', 'Riding on Exterior of Vehicle',
       'Second Seat, Other', 'Third Seat, Unknown',
       'Other Passenger in unenclosed passenger or cargo area',
       'Third Seat, Other', 'Trailing Unit',
       'Other Passenger in passenger or cargo area, unknown whether or not enclosed',
       'Fourth Seat, Left Side', 'Fourth Seat, Right Side',
       'Fourth Seat, Middle', 'Fourth Seat, Other'], dtype=object)

In [50]:
avp_df['seat_imname']=avp_df['seat_imname'].replace([
    'Front Seat, Left Side', 
    
    'Front Seat, Right Side',
    'Front Seat, Middle',
    'Front Seat, Unknown',
    'Front Seat, Other', 
    'Not Reported',
    'Reported as Unknown', 
    
    'Second Seat, Left Side', 
    'Second Seat, Middle',
    'Second Seat, Right Side', 
    'Second Seat, Unknown', 
    'Second Seat, Other',
    
    'Third Seat, Left Side',
    'Third Seat, Middle', 
    'Third Seat, Right Side', 
    'Third Seat, Unknown', 
    'Third Seat, Other',
    'Fourth Seat, Left Side',
    'Fourth Seat, Right Side', 
    'Fourth Seat, Middle',
    'Fourth Seat, Other',
    
    'Riding on Exterior of Vehicle', 
    'Appended to a Motor Vehicle for Motion',
    
    'Other Passenger in enclosed passenger or cargo area',
    'Sleeper Section of Cab (Truck)',
    'Other Passenger in passenger or cargo area, unknown whether or not enclosed',
    'Other Passenger in unenclosed passenger or cargo area',
    
    'Trailing Unit',
    
],[
    'Driver',
    
    'Front_passenger',
    'Front_passenger',
    'Front_passenger',
    'Front_passenger',
    'Front_passenger',
    'Front_passenger',
    
    'Second_row',
    'Second_row',
    'Second_row',
    'Second_row',
    'Second_row',
    
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    
    'Riding_outside',
    'Riding_outside',
    
    'Cargo_area',
    'Cargo_area',
    'Cargo_area',
    'Cargo_area',
    
    'Trailer'    
])
avp_df['seat_imname'].unique()

array(['Second_row', 'Driver', 'Front_passenger', 'Riding_outside',
       'Third_or_4 Row', 'Cargo_area', 'Trailer'], dtype=object)

In [51]:
avp_df['age_im_temp']=avp_df['age_im'].replace([997,998,999],[np.NaN,np.NaN,np.NaN])
avp_df['age_im_temp'].describe()

count    238805.000000
mean         37.247930
std          19.046642
min           0.000000
25%          22.000000
50%          34.000000
75%          51.000000
max         120.000000
Name: age_im_temp, dtype: float64

In [52]:
# Recode age - impute median age = 34 years for missing
avp_df['age_im']=avp_df['age_im'].replace([998,999],[34,34])
avp_df['age_im'].describe()

count    256440.000000
mean         37.024575
std          18.398439
min           0.000000
25%          23.000000
50%          34.000000
75%          50.000000
max         120.000000
Name: age_im, dtype: float64

In [53]:
# Write to CSV SEMI CLEAN - NEEDS SOME FIELDS DROPPED - NOT THE FINAL SET FOR MACHINE LEARNING
avp_df.to_csv('all_gas_no_brakes_semiclean.csv')

# NOT UPDATED BELOW THIS POINT

# FINAL PROJECT MACHINE LEARNING CODE


In [54]:
# CONNECT DATA TO DATABASE
crash_df = pd.read_csv("Resources/crash.csv")
crash_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Resources/crash.csv'

In [ ]:
# PREPROCESS THE DATA

In [ ]:
crash_df.columns

In [ ]:
# Check unique catagorical 
crash_df.nunique()

In [ ]:
# Look at XXXX value counts for binning
XXXX_counts = application_df.XXXX.value_counts()
#  How many XXXX counts are greater than X?
XXXX_counts[XXXX_counts>X]

In [ ]:
#  How many XXXX counts are less than or equal to X?
XXXX_counts[XXXX_counts <= X]

In [ ]:
# Visualize the value counts of XXXX
XXXX_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than or equal to X.
replace_crash = list(XXXX_counts[XXXX_counts <= X].index)

# Replace in dataframe
for crash in replace_crash:
    crash_df.XXXX = crash_df.XXXX.replace(app,"Other")
    
# Check to make sure binning was successful
crash_df.XXXX.value_counts()

In [ ]:
# Generate our categorical variable lists
crash_cat = crash_df.dtypes[crash_df.dtypes == "object"].index.tolist()

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(crash_df[crash_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(crash_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
crash_df = crash_df.merge(encode_df,left_index=True, right_index=True)
crash_df = crash_df.drop(crash_cat,1)
crash_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = crash_df["YVALUE"].values
X = crash_df.drop(["YVALUE"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  200
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("crash_checkpoints/",exist_ok=True)
checkpoint_path = "crash_checkpoints/weights.{epoch:02d}.hdf5"

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq='epoch')

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100, callbacks=[cp_callback])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn.save("Crash.h5")